# Description

In [27]:
"""
List of cars, regulated pollutants
NEDC hot 23 vs WLTP hot 23

""" 

'\nList of cars, regulated pollutants\nNEDC hot 23 vs WLTP hot 23\n\n'

# Libraries

In [19]:
%matplotlib inline
import pandas as pd, datetime as dt
#from co2mpas.datasync import _get_interp_method
import collections
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import glob, os
import time

## Definitions

In [20]:
car = {'Audi A1': ['./Audi A1/', 'xls',98,'gasoline'],             # we could get vela2 PN data from the analysis_veh.xls DB
      'BMW 530d': ['./BMW 530d/', 'xlsx',124,'diesel'],
      'Audi A3': ['./Audi A3/', 'xls',120,'diesel'],               # specify the FWD for NEDC in function lab_a1 
      'Citroen Cactus': ['./Citroen Cactus/','xls',95,'diesel'],   # specify WLTP_COLD in function lab_a1
      'Fiat 500x': ['./Fiat 500x/','xls',144,'diesel'],
      'Opel Astra': ['./Opel Astra/','',103,'gasoline'],           # Alternative code because of Vela8 file format (.xls for WLTP and .xlsx for NEDC)
      'Fiat Panda': ['./Fiat Panda/', 'xlsx',119,'gasoline'],
      'Kia Sportage': ['./Kia Sportage/','xlsx',124,'diesel'],
      'Peugeot Partner': ['./Peugeot Partner/','xls',112,'diesel'],
      'Renault Twingo': ['./Renault Twingo/', 'xlsx',112,'gasoline'],
      'VW Golf': ['./VW Golf/','xlsx',117,'diesel'],
      'Porsche Cayenne': ['./Porsche Cayenne/','xlsx',179,'diesel'] # out 
      #'Renault Captur':[] 
      #'VW Tiguan': ['./VW Tiguan/','xlsx',159,'diesel'],          # this remains OUT. not enough info and it'd require changes 
      }

In [293]:
vehicle = 'Citroen Cactus'

In [294]:
path = car[vehicle][0]
filetype = car[vehicle][1]
t_a = car[vehicle][2]
fuel = car[vehicle][3]

## Create the empty dataframe (ONLY BEFORE THE 1ST CAR!!!)

In [48]:
"""
###############################
Only to be used the first time!
###############################
"""

arrays = [['fuel','T_A',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           ],
           ['-', '-', 
           'CO2','CO2','CO2','CO2','CO2',             # reorder columns
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN','PN','PN',
           ],
           ['-', '-',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','urban','extraurban','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           'complete','low','medium','high','extrahigh','300/d','300',
           ]
         ]

tuples = list(zip(*arrays))

my_columns = pd.MultiIndex.from_tuples(tuples, names=['cycle', 'pollutant','phase'])

my_index = []



In [ ]:
RegPollEmi = pd.DataFrame(index=my_index, columns=my_columns)
#RegPollEmi.columns

# Vehicle

## Lab

### Distance based

#### Only for A3 

In [63]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_HOT*FWD*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLTP_HOT*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

#### Only for Cactus

In [84]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_HOT*FW*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLTP_HOT*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

#### Most of cars

In [312]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_HOT_*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {                                                        #  _F*  for VW Golf
                     '300/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLT*HOT*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '300/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

#### Follow from here

In [313]:
def lab_A1(cycle, file, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = A1['cycle'][cycle]['phases'][phases][0]
        stop = A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=1, header=0)                          # read the proper sheets within each file
        print(time.time() - tic)
        
        v = df['DACtrl_ActualSpeed\n[kph]'][start:stop]                                  # speed (km/h)
        t_s = len(df.index[start:stop])                                                  # time(s)
        t_m = t_s/60                                                         # time (hours)
        t_h = t_s/3600                                                       # time (hours)
        v_mean = v.mean()                                                    # speed means
        d = v_mean * t_h                                                     # distance means
        
#         CO2_150 = df['c_CO2_Dil_g_vec\n[grams]'][0:150]
#         CO2_150d = CO2_150.sum()
        CO2 = df['c_CO2_Dil_g_vec\n[grams]'][start:stop]
        CO2_sum = CO2.sum()
        CO2km = CO2_sum/d                                                    # CO2 g/km 
        
        
        
        NOx = df['c_NOx_Dil_g_vec\n[grams]'][start:stop]
        NOx_sum = NOx.sum()*1000
        NOxkm = NOx_sum/d                                                    # NOx mg/km

        CO = df['c_CO_Dil_g_vec\n[grams]'][start:stop]
        CO_sum = CO.sum()*1000
        COkm = CO_sum/d                                                      # CO mg/km

        HC = df['c_THC_Dil_g_vec\n[grams]'][start:stop]
        HC_sum = HC.sum()*1000
        HCkm = HC_sum/d                                                      # HC mg/km

        flow_gas = df['c_Vmix_vec\n[m3/min]'][start:stop]                               # flow of gas mixture [m3/min]
        flow_avg = flow_gas.sum()/t_s                                       # average in m3/min
        vol_gas = flow_avg * t_m                                            # total volume of gas mixture (m3)
        vol_gas_cc = vol_gas * 1e6                                          # total volume of gas mixture (cm3)

        PN = df["Counter\n[cpcc]"][start:stop]                                         # column of PN flow in #/cm3
        try:
            PN_sum = PN.sum()                                                  
            PN_avg = PN_sum / t_s                                               # average of flow in #/cm3

            PN_total = PN_avg * vol_gas_cc                                      # total emitted in #
            PN_total_km = PN_total / d                                          # total emission in #/km
        except:
            PN_total_km = 0
        #key = each.split('T ')[1].split('.x')[0]                             # shortening the titles of the files

        #key = str(i) + "." + str(key)                                        # numbering the files
        dic_lab[each] = {
            'CO2km': CO2km,
            'NOxkm': NOxkm,
            'COkm': COkm,
            'HCkm': HCkm,
            'PNkm': PN_total_km,
            't_s': t_s, 
            'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

#### Alternative to handle vela8 files (NEDC Opel Astra)

In [140]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC*HOT_23*.xlsx',# + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '150/d': [0,300],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLT*HOT*23*.xls',# + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '150/d': [0,300],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

In [141]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


def lab_A2(cycle, file, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = A1['cycle'][cycle]['phases'][phases][0]
        stop = A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=0, header=None)

        print(time.time() - tic)
        
        
        vals = df.values
        mux = pd.MultiIndex.from_arrays(df.ffill(1).values[:2, 1:], names=[None, 'Time'])   # trick to create the proper MultiIndex
        df = pd.DataFrame(df.values[2:, 1:], df.values[2:, 0], mux)                         # same here

        df.drop(df.index[[0,1]],inplace=True)                                               # remove the 2 first rows after the headers

        CO2 = df['Diluted grams']['CO2'][start:stop]
        CO2_sum = CO2.sum()
        NOx = df['Diluted grams']['NOx'][start:stop]
        NOx_sum = NOx.sum()*1000                                                            # change unit to mg/km
        CO = df['Diluted grams']['CO-L'][start:stop]
        CO_sum = CO.sum()*1000
        HC = df['Diluted grams']['HC'][start:stop]
        HC_sum = HC.sum()*1000                                                            # change unit to mg/km
               
        PN = df['CVS']['PN']
        PN.columns = ['PN_cm3', 'PN_s']
        PN = PN.PN_s[start:stop]
        PN_sum = PN.sum()

        v = df['Chassis dyno']['Speed'][start:stop]                                   # group speed by intervals                    
        v_mean = v.mean()                                                  # speed means
        t_s = len(df.index[start:stop])                                                #  time(s)
        t_m = t_s/60
        t_h = t_s/3600                                                                    # time (hours)

        d = v_mean * t_h #b.reset_index(drop=True) * d.reset_index(drop=True)                # distance means  

        CO2km = CO2_sum/d                                                    # CO2 per km 
        NOxkm = NOx_sum/d                                                    # NOx / km                                                     
        COkm = CO_sum/d                                                    # NOx / km                                                  
        HCkm = HC_sum/d
        PNkm = PN_sum/d
        
        dic_lab[each] = {
                'CO2km': CO2km,
                'NOxkm': NOxkm,
                'COkm': COkm,
                'HCkm': HCkm,
                'PNkm': PNkm,
                't_s': t_s, 
                'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

In [142]:
dic_lab_all = {}

In [143]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    for phase in A1['cycle'][cycle]['phases']:
        if cycle == 'NEDC':
            #print(A1['cycle'][cycle])
            dic_lab_all[cycle][phase] = lab_A2(cycle, file, phase) 
            #[phase] = run_A1(cycle, file, phase)
        else:
            dic_lab_all[cycle][phase] = lab_A1(cycle, file, phase) 

5.079080581665039
4.793243408203125
4.989130735397339
4.809234142303467
4.8142359256744385
4.8362202644348145
4.719272136688232
4.794260263442993
0.650914192199707
0.6629195213317871
0.687330961227417
0.6989626884460449
0.7568173408508301
0.6458814144134521
0.7078971862792969
0.7368786334991455
0.721360445022583
0.6976749897003174
0.680842399597168
0.7412581443786621
0.7861073017120361
0.5746841430664062
0.6456289291381836
0.5466985702514648
0.5496914386749268
0.5487020015716553
0.5346920490264893
0.5447049140930176
0.5516653060913086
0.5466997623443604
0.5346987247467041
0.5516872406005859
0.5387599468231201
0.6446304321289062
0.5746691226959229
0.5466856956481934
0.5476858615875244
0.5456845760345459
0.5476856231689453
0.5316946506500244
0.5396904945373535
0.5446872711181641
0.5487022399902344
0.529700517654419
0.6396348476409912
0.5366756916046143
0.5426890850067139
0.5756680965423584
0.5437078475952148
0.5436692237854004
0.5447051525115967
0.549685001373291
0.5346744060516357
0.542

#### Follow from here

In [314]:
dic_lab_all = {}

In [315]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
        dic_lab_all[cycle][phase] = lab_A1(cycle, file, phase)

0.5227105617523193
0.5276825428009033
0.6506381034851074
0.5276968479156494
0.5306949615478516
0.5137045383453369
0.5247547626495361
0.5256972312927246
0.5466864109039307
0.5317091941833496
0.543668270111084
0.5256969928741455
0.783562421798706
0.8285367488861084
0.7905454635620117
0.8874893188476562
0.796541690826416
0.7915832996368408
0.7735695838928223
0.7825503349304199
0.7805588245391846
0.7645752429962158
0.7845332622528076
0.7885332107543945
0.775611400604248
0.9054794311523438
0.7815499305725098
0.7675571441650391
0.7835495471954346
0.7875480651855469


In [316]:
a = pd.DataFrame.from_dict({(i,j,k): dic_lab_all[i][j][k] 
                           for i in dic_lab_all.keys() 
                           for j in dic_lab_all[i].keys()
                           for k in dic_lab_all[i][j].keys()},)
                       #orient='index')
a = a.replace(0, np.nan)
a

NEDC  \
                                                                    300/d   
      ././Citroen Cactus\VELA2_01072016_002_Modal_TP_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.293259e+02                    
COkm                                                 NaN                    
HCkm                                        2.659276e-02                    
NOxkm                                       6.873613e+01                    
PNkm                                        3.821630e+09                    
d                                           1.380072e+00                    
t_s                                         3.000000e+02                    

                                                                        \
                                                                         
      ././Citroen Cactus\VELA2_05072016_002_Modal_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.288946e+02                 
COkm                                        2.861667e+01                 
HCkm                                        1.112145e-01                 
NOxkm                                       3.403529e+01                 
PNkm                                        3.358928e+09                 
d                                           1.385461e+00                 
t_s                                         3.000000e+02                 

                                                                           \
                                                                            
      ././Citroen Cactus\VELA2_29062016_003_Modal_TP_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.403932e+02                    
COkm                                        2.977965e+01                    
HCkm                                        1.203330e+00                    
NOxkm                                       1.640702e+02                    
PNkm                                        4.457704e+09                    
d                                           1.388477e+00                    
t_s                                         3.000000e+02                    

                                                                           \
                                                                 complete   
      ././Citroen Cactus\VELA2_01072016_002_Modal_TP_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.093249e+02                    
COkm                                        5.041161e-03                    
HCkm                                        3.331064e-03                    
NOxkm                                       5.311686e+01                    
PNkm                                        1.704076e+09                    
d                                           1.101748e+01                    
t_s                                         1.180000e+03                    

                                                                        \
                                                                         
      ././Citroen Cactus\VELA2_05072016_002_Modal_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.088231e+02                 
COkm                                        5.511582e+00                 
HCkm                                        1.397108e-02                 
NOxkm                                       9.073440e+00                 
PNkm                                        1.429633e+09                 
d                                           1.102874e+01                 
t_s                                         1.180000e+03                 

                                                                           \
                                                                            
      ././Citroen Cactus\VELA2_29062016_003_Modal_TP_NEDC_HOT_FWD_23C.xls   
CO2km                                      

In [317]:
# a.groupby(level=1).mean(axis=1)
a_avg =a.T.groupby(level=[0,1]).mean()       # mean of repeated tests
a_avg

CO2km       COkm      HCkm       NOxkm          PNkm  \
NEDC 300/d       132.871236  29.198158  0.447046   88.947215  3.879421e+09   
     complete    110.931169   3.088859  0.162121   73.270918  1.782654e+09   
     extraurban  104.500018   1.522624  0.047877   62.646913  9.393056e+08   
     urban       121.923060   9.945837  0.411522   91.417873  3.224152e+09   
WLTP 300/d       138.652321  16.211037  0.992131  142.544730  3.967703e+09   
     complete    141.986333   3.411325  0.137017   32.278654  1.417175e+09   
     extrahigh   167.679132   4.836599  0.566276   38.128206  7.907469e+08   
     high        125.081136   0.855931  0.008119    1.154597  9.974831e+08   
     low         137.211350  10.666440  0.694232   99.793339  4.034627e+09   
     medium      126.032417   0.057758  0.100226   25.003671  1.427773e+09   

                         d     t_s  
NEDC 300/d        1.384670   300.0  
     complete    11.024197  1180.0  
     extraurban   6.955590   400.0  
     urban        4.068607   780.0  
WLTP 300/d        2.042658   300.0  
     complete    23.282324  1800.0  
     extrahigh    8.249221   323.0  
     high         7.163404   455.0  
     low          3.104461   589.0  
     medium       4.765238   433.0

In [318]:
a_avg = a_avg.drop(['d','t_s'], axis=1)      # remove the columns not to be used
a_avg = a_avg.unstack(level=[-1,0])          # put the 2 rows in 1 row (larger)
a_avg = pd.DataFrame(a_avg).T
a_avg = a_avg.swaplevel(0,2,axis=1)          # order Multiindex levels
a_avg = a_avg.swaplevel(1,2,axis=1)          # order Multiindex levels
a_avg = a_avg.sort_index(axis=1, level=0)    # sort by pollutant
# a_avg[a_avg.columns.set_levels([['complete', 'urban','extraurban','150/d','complete',
#                                  'low','medium','high','extrahigh','150/d']], level=2)]
LAB1 = a_avg

In [319]:
LAB1.index = [vehicle]                             # index name
LAB1.columns = LAB1.columns.droplevel(level=[0,1])    # no interested in Multiindex yet, so convert to regular df

LAB1.columns =['CO2_300/d_N','CO2_COMP_N','CO2_EU_N','CO2_URB_N',          # rename columns
               'CO_300/d_N','CO_COMP_N','CO_EU_N','CO_URB_N',
               'HC_300/d_N','HC_COMP_N','HC_EU_N','HC_URB_N',
               'NOx_300/d_N','NOx_COMP_N','NOx_EU_N','NOx_URB_N',
               'PN_300/d_N','PN_COMP_N','PN_EU_N','PN_URB_N',
               'CO2_300/d_W','CO2_COMP_W','CO2_EH_W','CO2_HIGH_W','CO2_LOW_W','CO2_MED_W',
               'CO_300/d_W','CO_COMP_W','CO_EH_W','CO_HIGH_W','CO_LOW_W','CO_MED_W',
               'HC_300/d_W','HC_COMP_W','HC_EH_W','HC_HIGH_W','HC_LOW_W','HC_MED_W',
               'NOx_300/d_W','NOx_COMP_W','NOx_EH_W','NOx_HIGH_W','NOx_LOW_W','NOx_MED_W',
               'PN_300/d_W','PN_COMP_W','PN_EH_W','PN_HIGH_W','PN_LOW_W','PN_MED_W']
              
LAB1 = LAB1[['CO2_COMP_N','CO2_URB_N','CO2_EU_N','CO2_300/d_N',             # reorder columns
            'NOx_COMP_N','NOx_URB_N','NOx_EU_N','NOx_300/d_N',
            'CO_COMP_N','CO_URB_N','CO_EU_N','CO_300/d_N',
            'HC_COMP_N','HC_URB_N','HC_EU_N','HC_300/d_N',
            'PN_COMP_N','PN_URB_N','PN_EU_N','PN_300/d_N',
            'CO2_COMP_W','CO2_LOW_W','CO2_MED_W','CO2_HIGH_W','CO2_EH_W','CO2_300/d_W',
            'NOx_COMP_W','NOx_LOW_W','NOx_MED_W','NOx_HIGH_W','NOx_EH_W','NOx_300/d_W',
            'CO_COMP_W','CO_LOW_W','CO_MED_W','CO_HIGH_W','CO_EH_W','CO_300/d_W',
            'HC_COMP_W','HC_LOW_W','HC_MED_W','HC_HIGH_W','HC_EH_W','HC_300/d_W',
            'PN_COMP_W','PN_LOW_W','PN_MED_W','PN_HIGH_W','PN_EH_W','PN_300/d_W']]

LAB1

,CO2_COMP_N,CO2_URB_N,CO2_EU_N,CO2_300/d_N,NOx_COMP_N,NOx_URB_N,NOx_EU_N,NOx_300/d_N,CO_COMP_N,CO_URB_N,...,HC_MED_W,HC_HIGH_W,HC_EH_W,HC_300/d_W,PN_COMP_W,PN_LOW_W,PN_MED_W,PN_HIGH_W,PN_EH_W,PN_300/d_W
Citroen Cactus,110.931169,121.92306,104.500018,132.871236,73.270918,91.417873,62.646913,88.947215,3.088859,9.945837,...,0.100226,0.008119,0.566276,0.992131,1.417175e+09,4.034627e+09,1.427773e+09,9.974831e+08,7.907469e+08,3.967703e+09


### 300 sec (non-distance based)

In [320]:
def lab_A1(cycle, file):#, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = 0 #A1['cycle'][cycle]['phases'][phases][0]
        stop = 300 #A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        df = pd.read_excel(each, sheetname=1, header=0)                          # read the proper sheets within each file
        print(time.time() - tic)
        
        v = df['DACtrl_ActualSpeed\n[kph]'][start:stop]                                  # speed (km/h)
        t_s = len(df.index[start:stop])                                                  # time(s)
        t_m = t_s/60                                                         # time (hours)
        t_h = t_s/3600                                                       # time (hours)
        v_mean = v.mean()                                                    # speed means
        d = v_mean * t_h                                                     # distance means
        
#         CO2_150 = df['c_CO2_Dil_g_vec\n[grams]'][0:150]
#         CO2_150d = CO2_150.sum()
        CO2 = df['c_CO2_Dil_g_vec\n[grams]'][start:stop]
        CO2_sum = CO2.sum()
        CO2km = CO2_sum/d                                                    # CO2 g/km 
        
        
        
        NOx = df['c_NOx_Dil_g_vec\n[grams]'][start:stop]
        NOx_sum = NOx.sum()*1000
        NOxkm = NOx_sum/d                                                    # NOx mg/km

        CO = df['c_CO_Dil_g_vec\n[grams]'][start:stop]
        CO_sum = CO.sum()*1000
        COkm = CO_sum/d                                                      # CO mg/km

        HC = df['c_THC_Dil_g_vec\n[grams]'][start:stop]
        HC_sum = HC.sum()*1000
        HCkm = HC_sum/d                                                      # HC mg/km

        flow_gas = df['c_Vmix_vec\n[m3/min]'][start:stop]                               # flow of gas mixture [m3/min]
        flow_avg = flow_gas.sum()/t_s                                       # average in m3/min
        vol_gas = flow_avg * t_m                                            # total volume of gas mixture (m3)
        vol_gas_cc = vol_gas * 1e6                                          # total volume of gas mixture (cm3)

        PN = df["Counter\n[cpcc]"][start:stop]                                         # column of PN flow in #/cm3
        try:
            PN_sum = PN.sum()                                                  
            PN_avg = PN_sum / t_s                                               # average of flow in #/cm3

            PN_total = PN_avg * vol_gas_cc                                      # total emitted in #
            PN_total_km = PN_total / d                                          # total emission in #/km
        except:
            PN_total = 0
            PN_total_km = 0

        #key = each.split('T ')[1].split('.x')[0]                             # shortening the titles of the files

        #key = str(i) + "." + str(key)                                        # numbering the files
        dic_lab[each] = {
            'CO2km': CO2_sum,
            'NOxkm': NOx_sum,
            'COkm': CO_sum,
            'HCkm': HC_sum,
            'PNkm': PN_total,
            't_s': t_s, 
            'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

#### Only for Astra

In [149]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
def lab_A2(cycle, file):#, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = 0 #A1['cycle'][cycle]['phases'][phases][0]
        stop = 300 #A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=0, header=None)

        print(time.time() - tic)
        
        
        vals = df.values
        mux = pd.MultiIndex.from_arrays(df.ffill(1).values[:2, 1:], names=[None, 'Time'])   # trick to create the proper MultiIndex
        df = pd.DataFrame(df.values[2:, 1:], df.values[2:, 0], mux)                         # same here

        df.drop(df.index[[0,1]],inplace=True)                                               # remove the 2 first rows after the headers

        CO2 = df['Diluted grams']['CO2'][start:stop]
        CO2_sum = CO2.sum()
        NOx = df['Diluted grams']['NOx'][start:stop]
        NOx_sum = NOx.sum()*1000                                                            # change unit to mg/km
        CO = df['Diluted grams']['CO-L'][start:stop]
        CO_sum = CO.sum()*1000
        HC = df['Diluted grams']['HC'][start:stop]
        HC_sum = HC.sum()*1000                                                            # change unit to mg/km
               
        PN = df['CVS']['PN']
        PN.columns = ['PN_cm3', 'PN_s']
        PN = PN.PN_s[start:stop]
        PN_sum = PN.sum()

        v = df['Chassis dyno']['Speed'][start:stop]                                   # group speed by intervals                    
        v_mean = v.mean()                                                  # speed means
        t_s = len(df.index[start:stop])                                                #  time(s)
        t_m = t_s/60
        t_h = t_s/3600                                                                    # time (hours)

        d = v_mean * t_h #b.reset_index(drop=True) * d.reset_index(drop=True)                # distance means  

        CO2km = CO2_sum/d                                                    # CO2 per km 
        NOxkm = NOx_sum/d                                                    # NOx / km                                                     
        COkm = CO_sum/d                                                    # NOx / km                                                  
        HCkm = HC_sum/d
        PNkm = PN_sum/d

        dic_lab[each] = {
                'CO2km': CO2_sum,
                'NOxkm': NOx_sum,
                'COkm': CO_sum,
                'HCkm': HC_sum,
                'PNkm': PN_sum,
                't_s': t_s, 
                'd': d}                                         # save means in the dictionary
        i = i+1

    return dic_lab
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


'\nONLY FOR ASTRA!!\nONLY FOR ASTRA!!\nONLY FOR ASTRA!!\n'

In [150]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""

for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
    if cycle == 'NEDC':
            #print(A1['cycle'][cycle])
        dic_lab_all[cycle] = lab_A2(cycle, file) 
            #[phase] = run_A1(cycle, file, phase)
    else:
        dic_lab_all[cycle] = lab_A1(cycle, file)

5.087088346481323
4.801238775253296
0.5696730613708496
0.5426785945892334
0.5477423667907715
0.6176614761352539
0.5486831665039062
0.5346939563751221
0.5436863899230957
0.5396723747253418
0.5447049140930176
0.5266966819763184
0.5396902561187744
0.5376770496368408
0.5436968803405762


#### Follow from here

In [321]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_lab_all[cycle]= lab_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

0.5217137336730957
0.5276963710784912
0.5157032012939453
0.7695572376251221
0.7805526256561279
0.9144747257232666


In [322]:
b = pd.DataFrame.from_dict({(i,j): dic_lab_all[i][j] 
                           for i in dic_lab_all.keys() 
                           for j in dic_lab_all[i].keys()},)
                       #orient='index')
b = b.replace(0,np.nan)
b

NEDC  \
      ././Citroen Cactus\VELA2_01072016_002_Modal_TP_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.784791e+02                    
COkm                                                 NaN                    
HCkm                                        3.669994e-02                    
NOxkm                                       9.486084e+01                    
PNkm                                        5.274127e+09                    
d                                           1.380072e+00                    
t_s                                         3.000000e+02                    

                                                                        \
      ././Citroen Cactus\VELA2_05072016_002_Modal_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.785785e+02                 
COkm                                        3.964729e+01                 
HCkm                                        1.540834e-01                 
NOxkm                                       4.715458e+01                 
PNkm                                        4.653664e+09                 
d                                           1.385461e+00                 
t_s                                         3.000000e+02                 

                                                                           \
      ././Citroen Cactus\VELA2_29062016_003_Modal_TP_NEDC_HOT_FWD_23C.xls   
CO2km                                       1.949328e+02                    
COkm                                        4.134837e+01                    
HCkm                                        1.670796e+00                    
NOxkm                                       2.278078e+02                    
PNkm                                        6.189422e+09                    
d                                           1.388477e+00                    
t_s                                         3.000000e+02                    

                                                                     WLTP  \
      ././Citroen Cactus\VELA2_01072016_004_Modal_TP_WLTP_HOT_FWD_23C.xls   
CO2km                                       2.860676e+02                    
COkm                                        2.518684e+01                    
HCkm                                        5.217664e-01                    
NOxkm                                       1.172353e+02                    
PNkm                                        6.076700e+09                    
d                                           2.049744e+00                    
t_s                                         3.000000e+02                    

                                                                        \
      ././Citroen Cactus\VELA2_06072016_003_Modal_WLTP_HOT_FWD_23C.xls   
CO2km                                       2.753642e+02                 
COkm                                        3.113285e+00                 
HCkm                                                 NaN                 
NOxkm                                       5.730024e+02                 
PNkm                                        1.282967e+10                 
d                                           2.037115e+00                 
t_s                                         3.000000e+02                 

                                                                           
      ././Citroen Cactus\VELA2_30062016_002_Modal_TP_WLTP_HOT_FWD_23C.xls  
CO2km                                       2.882477e+02                   
COkm                                        7.106553e+01                   
HCkm                                        3.530535e+00                   
NOxkm                                       1.819811e+02                   
PNkm                                        5.389634e+09                   
d                                           2.041114e+00                   
t_s                                         3

In [323]:
b_avg = b.T.groupby(level=0).mean()
b_avg = b_avg.drop(['d','t_s'], axis=1)
b_avg

,CO2km,COkm,HCkm,NOxkm,PNkm
NEDC,183.996803,40.497830,0.620527,123.274412,5.372404e+09
WLTP,283.226484,33.121885,2.026151,290.739627,8.098667e+09


In [324]:
NEDC300 = pd.DataFrame(b_avg.loc['NEDC',:])
NEDC300 = NEDC300.T
NEDC300.index = [vehicle]
WLTP300 = pd.DataFrame(b_avg.loc['WLTP',:])
WLTP300 = WLTP300.T
WLTP300.index = [vehicle]
WLTP300

LAB2 = pd.concat([NEDC300,WLTP300],axis=1)
LAB2.columns = ['CO2300_N','CO300_N','HC300_N','NOx300_N','PN300_N',
               'CO2300_W','CO300_W','HC300_W','NOx300_W','PN300_W']
LAB2 = LAB2[['CO2300_N','NOx300_N','CO300_N','HC300_N','PN300_N',
               'CO2300_W','NOx300_W','CO300_W','HC300_W','PN300_W']]
LAB2

,CO2300_N,NOx300_N,CO300_N,HC300_N,PN300_N,CO2300_W,NOx300_W,CO300_W,HC300_W,PN300_W
Citroen Cactus,183.996803,123.274412,40.49783,0.620527,5.372404e+09,283.226484,290.739627,33.121885,2.026151,8.098667e+09


In [308]:
LAB = pd.concat([LAB1,LAB2],axis=1)
LAB = LAB[[
           'CO2_COMP_N','CO2_URB_N','CO2_EU_N','CO2_300/d_N','CO2300_N',             # reorder columns
           'NOx_COMP_N','NOx_URB_N','NOx_EU_N','NOx_300/d_N','NOx300_N',
           'CO_COMP_N','CO_URB_N','CO_EU_N','CO_300/d_N','CO300_N',
           'HC_COMP_N','HC_URB_N','HC_EU_N','HC_300/d_N','HC300_N',
           'PN_COMP_N','PN_URB_N','PN_EU_N','PN_300/d_N','PN300_N',
           'CO2_COMP_W','CO2_LOW_W','CO2_MED_W','CO2_HIGH_W','CO2_EH_W','CO2_300/d_W','CO2300_W',
           'NOx_COMP_W','NOx_LOW_W','NOx_MED_W','NOx_HIGH_W','NOx_EH_W','NOx_300/d_W','NOx300_W',
           'CO_COMP_W','CO_LOW_W','CO_MED_W','CO_HIGH_W','CO_EH_W','CO_300/d_W','CO300_W',
           'HC_COMP_W','HC_LOW_W','HC_MED_W','HC_HIGH_W','HC_EH_W','HC_300/d_W','HC300_W',
           'PN_COMP_W','PN_LOW_W','PN_MED_W','PN_HIGH_W','PN_EH_W','PN_300/d_W','PN300_W'
    ]]
LAB

,CO2_COMP_N,CO2_URB_N,CO2_EU_N,CO2_300/d_N,CO2300_N,NOx_COMP_N,NOx_URB_N,NOx_EU_N,NOx_300/d_N,NOx300_N,...,HC_EH_W,HC_300/d_W,HC300_W,PN_COMP_W,PN_LOW_W,PN_MED_W,PN_HIGH_W,PN_EH_W,PN_300/d_W,PN300_W
Citroen Cactus,108.823076,118.291097,103.287588,128.894628,178.578524,9.07344,18.896587,3.330328,34.035294,47.154585,...,NaN,NaN,NaN,1.630656e+09,5.801314e+09,1.424159e+09,8.471410e+08,8.580410e+08,6.698815e+09,1.367419e+10


In [326]:
LAB.WLTP

pollutant              CO2                                                  \
phase             complete         low      medium        high   extrahigh   
Citroen Cactus  140.562926  133.837614  123.629375  123.821033  167.312694   

pollutant                                     NOx                         \
phase                300/d         300   complete         low     medium   
Citroen Cactus  134.573468  274.668743  55.138681  190.376868  56.398763   

pollutant           ...             HC                      PN                \
phase               ...      extrahigh 300/d 300      complete           low   
Citroen Cactus      ...            NaN   NaN NaN  1.630656e+09  5.801314e+09   

pollutant                                                               \
phase                 medium          high     extrahigh         300/d   
Citroen Cactus  1.424159e+09  8.471410e+08  8.580410e+08  6.698815e+09   

pollutant                     
phase                    300  
Citroen Cactus  1.367419e+10  

[1 rows x 35 columns]

## RDE

In [539]:
A1 = {
    'cycle': {
        'Esperia': {'file': './'+path+'/*ESP*.xlsx'},
        'Labiena': {'file': './'+path+'/*LAB*.xlsx'}, 
             },
    
     }

### Distance based

In [540]:
def road_A1(cycle, file):
    files_RDE = glob.glob(file)                                         # read all EMROAD files
    dic_RDE = {}                                                            # empty dictionary road test

    i = 1
    for each in files_RDE:

        tic = time.time()
        df = pd.read_excel(each, sheetname='TEST REPORT', header=6)             # read the proper sheets within each file
        print(time.time() - tic)
        
        df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)   # convert the headers to read them: ' ' for '_'
        df.rename(columns=lambda x: x.strip().replace(".", "_"), inplace=True)   # convert the headers to read them: '.' for '_'


        CO2_COMP = df.COMPLETE_TEST['CO2'][1]      # g/km
        CO2_URB = df.URBAN['CO2'][1]               # g/km
        CO2_RUR = df.RURAL['CO2'][1]               # g/km
        CO2_MOT = df.MOTORW_['CO2'][1]             # g/km
        #CO2 = [CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT]

        NOx_COMP = df.COMPLETE_TEST['NOx'][1]*1000
        NOx_URB = df.URBAN['NOx'][1]*1000
        NOx_RUR = df.RURAL['NOx'][1]*1000
        NOx_MOT = df.MOTORW_['NOx'][1]*1000

        CO_COMP = df.COMPLETE_TEST['CO'][1]*1000      # g/km
        CO_URB = df.URBAN['CO'][1]*1000               # g/km
        CO_RUR = df.RURAL['CO'][1]*1000               # g/km
        CO_MOT = df.MOTORW_['CO'][1]*1000             # g/km

        HC_COMP = df.COMPLETE_TEST['THC'][1]*1000      # g/km
        HC_URB = df.URBAN['THC'][1]*1000               # g/km
        HC_RUR = df.RURAL['THC'][1]*1000               # g/km
        HC_MOT = df.MOTORW_['THC'][1]*1000             # g/km

        PN_COMP = df.COMPLETE_TEST['PN'][1]      # g/km
        PN_URB = df.URBAN['PN'][1]               # g/km
        PN_RUR = df.RURAL['PN'][1]               # g/km
        PN_MOT = df.MOTORW_['PN'][1]             # g/km

        #key = each.split('T ')[1].split('.x')[0]
        ###########################################       each = str(i) + "_" + str(each)
        dic_RDE[each] = {'CO2_COMP':CO2_COMP,'CO2_URB':CO2_URB,'CO2_RUR':CO2_RUR,'CO2_MOT':CO2_MOT,
                          'NOx_COMP':NOx_COMP,'NOx_URB':NOx_URB,'NOx_RUR':NOx_RUR,'NOx_MOT':NOx_MOT,
                          'CO_COMP':CO_COMP,'CO_URB':CO_URB,'CO_RUR':CO_RUR,'CO_MOT':CO_MOT,
                          'HC_COMP':HC_COMP,'HC_URB':HC_URB,'HC_RUR':HC_RUR,'HC_MOT':HC_MOT,
                          'PN_COMP':PN_COMP,'PN_URB':PN_URB,'PN_RUR':PN_RUR,'PN_MOT':PN_MOT,
                        }                                               # save means in the dictionary
        i = i+1
    return dic_RDE

In [541]:
dic_road_all = {}

In [542]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_road_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_road_all[cycle]= road_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

13.745081186294556
16.13997530937195
15.11614203453064
14.517651081085205
16.27964973449707
16.276635885238647
16.48651885986328
16.01880407333374


In [543]:
c = pd.DataFrame.from_dict({(i,j): dic_road_all[i][j] 
                           for i in dic_road_all.keys() 
                           for j in dic_road_all[i].keys()},)
                       #orient='index')

c = c.replace(0,np.nan)                  # We put NaN instead of 0 so that those are not considered in the .mean()
c.T

CO2_COMP  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  117.985222   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  119.551407   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  120.900589   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  127.296716   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  135.675119   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  135.587369   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  130.071683   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  154.382539   

                                                               CO2_MOT  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  135.051214   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  137.455698   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  132.463132   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  140.768259   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  143.987892   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  146.399864   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  148.504152   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  154.386947   

                                                               CO2_RUR  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  102.276445   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  102.052041   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  114.234442   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  121.686802   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  115.257651   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  119.834242   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  106.513911   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  152.352989   

                                                               CO2_URB  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...  114.999399   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...  117.193040   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  115.397952   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...  118.155502   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  141.472448   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...  133.318106   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  123.692578   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  155.424813   

                                                               CO_COMP  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...   31.436863   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...   46.693675   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...   71.097595   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...   16.524425   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...   64.180176   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...   16.350751   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...   83.273701   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  343.887267   

                                                                CO_MOT  \
Esperia ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_...         NaN   
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_...   19.918229   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...   41.951971   
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_...    2.853006   
Labiena ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...   31.195548   
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_...    0.338315   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...   61.814641   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_...  100.929100   

                                                            

In [544]:
c_avg = c.T.groupby(level=0).mean()
c_avg = c_avg[['CO2_COMP','CO2_URB','CO2_RUR','CO2_MOT',
              'NOx_COMP','NOx_URB','NOx_RUR','NOx_MOT',
              'CO_COMP','CO_URB','CO_RUR','CO_MOT',
              'HC_COMP','HC_URB','HC_RUR','HC_MOT',
              'PN_COMP','PN_URB','PN_RUR','PN_MOT']]
c_avg

,CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT,NOx_COMP,NOx_URB,NOx_RUR,NOx_MOT,CO_COMP,CO_URB,CO_RUR,CO_MOT,HC_COMP,HC_URB,HC_RUR,HC_MOT,PN_COMP,PN_URB,PN_RUR,PN_MOT
Esperia,121.433483,116.436473,110.062433,136.434575,419.648304,181.576488,526.155259,604.512179,41.438140,80.755037,23.280258,21.574402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Labiena,138.929177,138.476986,123.489698,148.319714,503.138979,361.526775,630.079824,557.238579,126.922974,208.927415,167.675826,48.569401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [545]:
RDE1 = pd.DataFrame(c_avg.mean())
RDE1.columns = [vehicle]
RDE1 = RDE1.T
RDE1

,CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT,NOx_COMP,NOx_URB,NOx_RUR,NOx_MOT,CO_COMP,CO_URB,CO_RUR,CO_MOT,HC_COMP,HC_URB,HC_RUR,HC_MOT,PN_COMP,PN_URB,PN_RUR,PN_MOT
Citroen Cactus,130.18133,127.45673,116.776065,142.377145,461.393642,271.551632,578.117541,580.875379,84.180557,144.841226,95.478042,35.071902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### RDE 300 sec (in g)

In [546]:
def road_A1(cycle, file):
    files_RDE = glob.glob(file)                                         # read all EMROAD files
    dic_RDE = {}                                                            # empty dictionary road test

    i = 1
    for each in files_RDE:

        tic = time.time()
        df = pd.read_excel(each, sheetname='CALCULATED DATA', header=1,skiprows=[2])
        print(time.time() - tic)
        
        df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)   # convert the headers to read them: ' ' for '_'
        df.rename(columns=lambda x: x.strip().replace(".", "_"), inplace=True)   # convert the headers to read them: '.' for '_'
    
        min_ind = df.loc[(df["Inst__Mass_CO2"] > 0.1)].index[0] # minimum index, engine turned on
           #  & (df["Inst__Mass_NOx"] > 0.000001)           # threshold chosen by looking the files
    
        CO2_300 = df.Inst__Mass_CO2[min_ind:min_ind+300].sum()           # grams of CO2
        NOx_300 = df.Inst__Mass_NOx[min_ind:min_ind+300].sum()*1000      # miligrams of NOx
        CO_300 = df.Inst__Mass_CO[min_ind:min_ind+300].sum()*1000       # miligrams of CO
        HC_300 = df.Inst__Mass_THC[min_ind:min_ind+300].sum()*1000      # miligrams of HC
        PN_300 = df.Inst__PN[min_ind:min_ind+300].sum()                 # total PN
        
        dic_RDE[each] = {
            'CO2 300': CO2_300,
            'NOx 300': NOx_300,
            'CO 300': CO_300,
            'HC 300': HC_300,
            'PN 300': PN_300
        }                                              # save means in the dictionary
        
        i = i+1
        
    return dic_RDE

In [547]:
dic_road_all = {}

In [548]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_road_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_road_all[cycle]= road_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

15.226254940032959
15.251230001449585
15.051345348358154
14.588626146316528
16.339602947235107
16.634414672851562
16.187685251235962
26.624704837799072


In [549]:
d = pd.DataFrame.from_dict({(i,j): dic_road_all[i][j] 
                           for i in dic_road_all.keys() 
                           for j in dic_road_all[i].keys()},)
                       #orient='index')
d = d.replace(0,np.nan)
d

Esperia  \
        ././Citroen Cactus\CN002_ESP_2016_06_16_EMROAD_Test1.xlsx   
CO 300                                         2394.029267          
CO2 300                                         177.180875          
HC 300                                                 NaN          
NOx 300                                         449.348674          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_ESP_2016_06_17_EMROAD_Test1.xlsx   
CO 300                                         2041.136560          
CO2 300                                         261.217826          
HC 300                                                 NaN          
NOx 300                                         453.699495          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_Test1.xlsx   
CO 300                                         1349.738724          
CO2 300                                         292.811218          
HC 300                                                 NaN          
NOx 300                                         527.002472          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_ESP_2016_06_20_EMROAD_Test2.xlsx   
CO 300                                          535.569800          
CO2 300                                         225.616876          
HC 300                                                 NaN          
NOx 300                                         664.026639          
PN 300                                                 NaN          

                                                          Labiena  \
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_Test1.xlsx   
CO 300                                         1502.704877          
CO2 300                                         280.564514          
HC 300                                                 NaN          
NOx 300                                         558.979506          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_LAB_2016_06_24_EMROAD_Test2.xlsx   
CO 300                                          427.992143          
CO2 300                                         293.004226          
HC 300                                                 NaN          
NOx 300                                         808.017974          
PN 300                                                 NaN          

                                                                   \
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_Test1.xlsx   
CO 300                                          900.411417          
CO2 300                                         271.632339          
HC 300                                                 NaN          
NOx 300                                         676.741891          
PN 300                                                 NaN          

                                                                   
        ././Citroen Cactus\CN002_LAB_2016_06_28_EMROAD_Test2.xlsx  
CO 300                                          254.829324         
CO2 300                                         325.023221         
HC 300                                                 NaN         
NOx 300                                        1037.343284         
PN 300                                                 NaN

In [550]:
d_avg = d.T.groupby(level=0).mean()
d_avg = d_avg[['CO2 300','NOx 300','CO 300','HC 300','PN 300']]
d_avg

,CO2 300,NOx 300,CO 300,HC 300,PN 300
Esperia,239.206699,523.519320,1580.118588,NaN,NaN
Labiena,292.556075,770.270664,771.484440,NaN,NaN


In [551]:
e = pd.DataFrame(d_avg.mean())
e.columns = [vehicle]
RDE2 = e.T
RDE2

,CO2 300,NOx 300,CO 300,HC 300,PN 300
Citroen Cactus,265.881387,646.894992,1175.801514,NaN,NaN


In [552]:
RDE = pd.concat([RDE1,RDE2],axis=1)
RDE = RDE[['CO2_COMP','CO2_URB','CO2_RUR','CO2_MOT','CO2 300',
             'NOx_COMP','NOx_URB','NOx_RUR','NOx_MOT','NOx 300',
             'CO_COMP','CO_URB','CO_RUR','CO_MOT','CO 300',
             'HC_COMP','HC_URB','HC_RUR','HC_MOT','HC 300',
             'PN_COMP','PN_URB','PN_RUR','PN_MOT','PN 300']]
RDE

,CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT,CO2 300,NOx_COMP,NOx_URB,NOx_RUR,NOx_MOT,NOx 300,...,HC_COMP,HC_URB,HC_RUR,HC_MOT,HC 300,PN_COMP,PN_URB,PN_RUR,PN_MOT,PN 300
Citroen Cactus,130.18133,127.45673,116.776065,142.377145,265.881387,461.393642,271.551632,578.117541,580.875379,646.894992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concat lab and road

In [553]:
veh = pd.concat([LAB,RDE], axis=1)#, ignore_index=True)
veh.insert(0, "T_A", t_a)
veh.insert(0,"fuel",fuel)
# AUDI_A1.columns =db_regpoll.columns
#veh.drop('Peugeot Partner', inplace=True)
veh

,fuel,T_A,CO2_COMP_N,CO2_URB_N,CO2_EU_N,CO2_300/d_N,CO2300_N,NOx_COMP_N,NOx_URB_N,NOx_EU_N,...,HC_COMP,HC_URB,HC_RUR,HC_MOT,HC 300,PN_COMP,PN_URB,PN_RUR,PN_MOT,PN 300
Citroen Cactus,diesel,95,114.342562,130.942092,104.667121,149.797212,206.448435,29.15528,54.059993,14.636739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Appending

In [325]:
LAB.insert(0, "T_A", t_a)
LAB.insert(0,"fuel",fuel)
LAB

ValueError: cannot insert T_A, already exists

In [310]:
LAB.columns = my_columns
RegPollEmi = RegPollEmi.append(LAB)

In [311]:
RegPollEmi                             #.sort_index()#.drop('Peugeot Partner', inplace=True)

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  109.224749  126.261639   99.281020   
BMW 530d           diesel  124  136.646383  167.780078  118.550458   
Audi A3            diesel  120  143.617851  159.482966  134.529921   
Citroen Cactus     diesel   95  110.931169  121.923060  104.500018   
Fiat 500x          diesel  144  178.252771  187.425297  172.986949   
Opel Astra       gasoline  103  117.819303  137.228226  106.493130   
Fiat Panda       gasoline  119  122.897700  152.522899  105.576738   
Kia Sportage       diesel  124  123.500111  141.036546  113.295535   
Peugeot Partner    diesel  112  110.110581  116.389014  106.463950   
Renault Twingo   gasoline  112  113.911697  140.996459   98.036526   
VW Golf            diesel  117  112.787399  132.706869  101.488632   
Peugeot Partner    diesel  112  111.051605  118.873478  106.495967   
Citroen Cactus     diesel   95  108.823076  118.291097  103.287588   

cycle                                                                        \
pollutant                                       NOx                           
phase                 300/d         300    complete       urban  extraurban   
Audi A1          131.674431  182.238201    3.244371    3.853328    2.888679   
BMW 530d         192.567466  265.926267   11.813196    6.378457   14.972416   
Audi A3          166.659848  224.658617   87.233923  106.952738   75.938499   
Citroen Cactus   132.871236  183.996803   73.270918   91.417873   62.646913   
Fiat 500x        204.157466  277.520157  255.537778   76.237919  358.497340   
Opel Astra       151.206719  208.728917    8.559600   18.119637    2.982083   
Fiat Panda       162.703650  225.174072    3.257790    1.198152    4.461632   
Kia Sportage     155.932138  214.588194   35.102620   37.430163   33.748206   
Peugeot Partner  123.859836  169.796308   14.244487   32.194838    3.867223   
Renault Twingo   147.476091  205.010291    5.364229    8.625508    3.452672   
VW Golf          152.549200  203.449542   26.601251   30.008626   24.674258   
Peugeot Partner  127.420470  175.151652   14.602113   29.752761    5.832007   
Citroen Cactus   128.894628  178.578524    9.073440   18.896587    3.330328   

cycle                ...            WLTP                          \
pollutant            ...              HC                           
phase                ...       extrahigh       300/d         300   
Audi A1              ...        4.145784   39.598324   81.033869   
BMW 530d             ...       14.392196    1.987197    4.020022   
Audi A3              ...       26.756063         NaN         NaN   
Citroen Cactus       ...        0.566276    0.992131    2.026151   
Fiat 500x            ...       12.709592    0.024629    0.049830   
Opel Astra           ...        9.143097   61.770894  125.922912   
Fiat Panda           ...        7.769349  121.506651  248.019954   
Kia Sportage         ...        9.609554    1.967097    3.993933   
Peugeot Partner      ...             NaN         NaN         NaN   
Renault Twingo       ...       10.592574   87.147733  180.289611   
VW Golf              ...        9.381191    0.696927    1.410645   
Peugeot Partner      ...             NaN         NaN         NaN   
Citroen Cactus       ...             NaN         NaN         NaN   

cycle                                                                    \
pollutant                  PN                                             
phase                complete           low        medium          high   
Audi A1                   NaN           NaN           NaN           NaN   
BMW 530d         1.748835e+10  4.942393e+10  2.162154e+10  1.448308e+10   
Audi A3          6.676275e+09  4.262223e+10  1.863999e+09  1.241815e+09   
Citroen Cactus   1.492013e+09  4.545278e+09  1.468726e+09  9.476009e+08   


# Excel and DataFrame

In [327]:
RegPollEmi.to_excel('veh_hot.xlsx')

PermissionError: [Errno 13] Permission denied: 'veh_hot.xlsx'

In [330]:
hot = pd.read_excel('veh_hot.xlsx',header=(0,1,2))
hot

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  109.224749  126.261639   99.281020   
BMW 530d           diesel  124  136.646383  167.780078  118.550458   
Audi A3            diesel  120  143.617851  159.482966  134.529921   
Citroen Cactus     diesel   95  110.931169  121.923060  104.500018   
Fiat 500x          diesel  144  178.252771  187.425297  172.986949   
Opel Astra       gasoline  103  117.819303  137.228226  106.493130   
Fiat Panda       gasoline  119  122.897700  152.522899  105.576738   
Kia Sportage       diesel  124  123.500111  141.036546  113.295535   
Peugeot Partner    diesel  112  111.051605  118.873478  106.495967   
Renault Twingo   gasoline  112  113.911697  140.996459   98.036526   
VW Golf            diesel  117  112.787399  132.706869  101.488632   

cycle                                                                        \
pollutant                                       NOx                           
phase                 300/d         300    complete       urban  extraurban   
Audi A1          131.674431  182.238201    3.244371    3.853328    2.888679   
BMW 530d         192.567466  265.926267   11.813196    6.378457   14.972416   
Audi A3          166.659848  224.658617   87.233923  106.952738   75.938499   
Citroen Cactus   132.871236  183.996803   73.270918   91.417873   62.646913   
Fiat 500x        204.157466  277.520157  255.537778   76.237919  358.497340   
Opel Astra       151.206719  208.728917    8.559600   18.119637    2.982083   
Fiat Panda       162.703650  225.174072    3.257790    1.198152    4.461632   
Kia Sportage     155.932138  214.588194   35.102620   37.430163   33.748206   
Peugeot Partner  127.420470  175.151652   14.602113   29.752761    5.832007   
Renault Twingo   147.476091  205.010291    5.364229    8.625508    3.452672   
VW Golf          152.549200  203.449542   26.601251   30.008626   24.674258   

cycle                ...            WLTP                          \
pollutant            ...              HC                           
phase                ...       extrahigh       300/d         300   
Audi A1              ...        4.145784   39.598324   81.033869   
BMW 530d             ...       14.392196    1.987197    4.020022   
Audi A3              ...       26.756063         NaN         NaN   
Citroen Cactus       ...        0.566276    0.992131    2.026151   
Fiat 500x            ...       12.709592    0.024629    0.049830   
Opel Astra           ...        9.143097   61.770894  125.922912   
Fiat Panda           ...        7.769349  121.506651  248.019954   
Kia Sportage         ...        9.609554    1.967097    3.993933   
Peugeot Partner      ...             NaN         NaN         NaN   
Renault Twingo       ...       10.592574   87.147733  180.289611   
VW Golf              ...        9.381191    0.696927    1.410645   

cycle                                                                    \
pollutant                  PN                                             
phase                complete           low        medium          high   
Audi A1                   NaN           NaN           NaN           NaN   
BMW 530d         1.748835e+10  4.942393e+10  2.162154e+10  1.448308e+10   
Audi A3          6.676275e+09  4.262223e+10  1.863999e+09  1.241815e+09   
Citroen Cactus   1.492013e+09  4.545278e+09  1.468726e+09  9.476009e+08   
Fiat 500x        1.367052e+09  3.518482e+09  1.584749e+09  1.016139e+09   
Opel Astra       1.716933e+12  2.090907e+12  1.471504e+12  1.195032e+12   
Fiat Panda       1.711442e+11  5.546561e+11  2.006076e+11  9.300220e+10   
Kia Sportage     2.247191e+09  9.126702e+09  2.079476e+09  1.120749e+09   
Peugeot Partner  8.039542e+09  2.976910e+10  7.282785e+09  4.442872e+09   
Renault Twingo   1.500027e+11  3.011436e+11  1.271117e+11  

### Manual changes?

In [331]:
#if values changed manually in the veh_hot Excel, then run this:
                    
hot = pd.read_excel('veh_hot.xlsx',header=(0,1,2)) 
RegPollEmi = hot     

In [332]:
RegPollEmi

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  109.224749  126.261639   99.281020   
BMW 530d           diesel  124  136.646383  167.780078  118.550458   
Audi A3            diesel  120  143.617851  159.482966  134.529921   
Citroen Cactus     diesel   95  110.931169  121.923060  104.500018   
Fiat 500x          diesel  144  178.252771  187.425297  172.986949   
Opel Astra       gasoline  103  117.819303  137.228226  106.493130   
Fiat Panda       gasoline  119  122.897700  152.522899  105.576738   
Kia Sportage       diesel  124  123.500111  141.036546  113.295535   
Peugeot Partner    diesel  112  111.051605  118.873478  106.495967   
Renault Twingo   gasoline  112  113.911697  140.996459   98.036526   
VW Golf            diesel  117  112.787399  132.706869  101.488632   

cycle                                                                        \
pollutant                                       NOx                           
phase                 300/d         300    complete       urban  extraurban   
Audi A1          131.674431  182.238201    3.244371    3.853328    2.888679   
BMW 530d         192.567466  265.926267   11.813196    6.378457   14.972416   
Audi A3          166.659848  224.658617   87.233923  106.952738   75.938499   
Citroen Cactus   132.871236  183.996803   73.270918   91.417873   62.646913   
Fiat 500x        204.157466  277.520157  255.537778   76.237919  358.497340   
Opel Astra       151.206719  208.728917    8.559600   18.119637    2.982083   
Fiat Panda       162.703650  225.174072    3.257790    1.198152    4.461632   
Kia Sportage     155.932138  214.588194   35.102620   37.430163   33.748206   
Peugeot Partner  127.420470  175.151652   14.602113   29.752761    5.832007   
Renault Twingo   147.476091  205.010291    5.364229    8.625508    3.452672   
VW Golf          152.549200  203.449542   26.601251   30.008626   24.674258   

cycle                ...            WLTP                          \
pollutant            ...              HC                           
phase                ...       extrahigh       300/d         300   
Audi A1              ...        4.145784   39.598324   81.033869   
BMW 530d             ...       14.392196    1.987197    4.020022   
Audi A3              ...       26.756063         NaN         NaN   
Citroen Cactus       ...        0.566276    0.992131    2.026151   
Fiat 500x            ...       12.709592    0.024629    0.049830   
Opel Astra           ...        9.143097   61.770894  125.922912   
Fiat Panda           ...        7.769349  121.506651  248.019954   
Kia Sportage         ...        9.609554    1.967097    3.993933   
Peugeot Partner      ...             NaN         NaN         NaN   
Renault Twingo       ...       10.592574   87.147733  180.289611   
VW Golf              ...        9.381191    0.696927    1.410645   

cycle                                                                    \
pollutant                  PN                                             
phase                complete           low        medium          high   
Audi A1                   NaN           NaN           NaN           NaN   
BMW 530d         1.748835e+10  4.942393e+10  2.162154e+10  1.448308e+10   
Audi A3          6.676275e+09  4.262223e+10  1.863999e+09  1.241815e+09   
Citroen Cactus   1.492013e+09  4.545278e+09  1.468726e+09  9.476009e+08   
Fiat 500x        1.367052e+09  3.518482e+09  1.584749e+09  1.016139e+09   
Opel Astra       1.716933e+12  2.090907e+12  1.471504e+12  1.195032e+12   
Fiat Panda       1.711442e+11  5.546561e+11  2.006076e+11  9.300220e+10   
Kia Sportage     2.247191e+09  9.126702e+09  2.079476e+09  1.120749e+09   
Peugeot Partner  8.039542e+09  2.976910e+10  7.282785e+09  4.442872e+09   
Renault Twingo   1.500027e+11  3.011436e+11  1.271117e+11  

# Ratios

In [268]:
cold = pd.read_excel('veh_cold.xlsx',header=(0,1,2))
cold

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  115.010052  141.499340   99.544178   
BMW 530d           diesel  124  142.593498  183.514712  118.979382   
Audi A3            diesel  120  151.025535  177.388071  135.889201   
Citroen Cactus     diesel   95  114.342562  130.942092  104.667121   
Fiat 500x          diesel  144  191.497721  215.876307  177.539683   
Opel Astra       gasoline  103  130.524707  162.965505  111.592557   
Fiat Panda       gasoline  119  129.183095  169.412697  105.658152   
Kia Sportage       diesel  124  130.444277  157.244714  114.835875   
Peugeot Partner    diesel  112  116.523647  130.515602  108.385462   
Renault Twingo   gasoline  112  120.245945  155.692207   99.469361   
VW Golf            diesel  117  122.991381  152.002223  105.176443   

cycle                                                                       \
pollutant                                       NOx                          
phase                 300/d         300    complete      urban  extraurban   
Audi A1          170.124042  235.608907   11.489641  24.933212    3.640121   
BMW 530d         204.821367  280.360502   15.070664  20.150810   12.136080   
Audi A3          199.679152  269.557351   67.007396  66.526807   67.285514   
Citroen Cactus   149.797212  206.448435   29.155280  54.059993   14.636739   
Fiat 500x        243.125268  328.664990  196.017402  78.637758  263.499495   
Opel Astra       195.682618  271.416752   10.096388  25.069613    1.360553   
Fiat Panda       199.757413  276.063497    4.073624   7.087758    2.310302   
Kia Sportage     185.116731  254.598819   38.779593  54.625389   29.551978   
Peugeot Partner  145.856678  200.044691   13.569765  34.975806    1.119893   
Renault Twingo   180.784348  251.310183    4.597819   9.416676    1.773294   
VW Golf          178.102152  238.340758   17.106838  21.802389   14.144599   

cycle                ...            WLTP                          \
pollutant            ...              HC                           
phase                ...       extrahigh       300/d         300   
Audi A1              ...        3.612463  244.612639  502.309493   
BMW 530d             ...       17.093954   23.649936   48.288021   
Audi A3              ...        9.244835   10.228618   20.705317   
Citroen Cactus       ...        0.002157    8.781724   17.798214   
Fiat 500x            ...        9.963727    7.618207   15.360624   
Opel Astra           ...        8.735902  327.515154  667.716128   
Fiat Panda           ...        8.315068  187.799907  384.632149   
Kia Sportage         ...       11.256764   38.203132   77.671689   
Peugeot Partner      ...        0.000000    6.900313   14.015382   
Renault Twingo       ...       11.724889  323.610715  664.626905   
VW Golf              ...       20.329904    0.638087    1.291352   

cycle                                                                    \
pollutant                  PN                                             
phase                complete           low        medium          high   
Audi A1          2.370000e+12  4.222000e+12           NaN           NaN   
BMW 530d         6.725514e+10  3.657574e+11  3.126323e+10  1.080758e+10   
Audi A3          8.788198e+11  5.899111e+12  2.487272e+11  9.350137e+10   
Citroen Cactus   3.835728e+09  2.254897e+10  1.435464e+09  9.188180e+08   
Fiat 500x        2.040206e+10  1.478100e+11  1.288385e+09  9.976248e+08   
Opel Astra       3.692108e+12  1.235003e+13  3.462407e+12  1.563244e+12   
Fiat Panda       2.630485e+11  1.231589e+12  2.129664e+11  8.849184e+10   
Kia Sportage     3.305123e+09  1.802407e+10  1.780748e+09  1.033948e+09   
Peugeot Partner  6.933058e+09  3.128800e+10  5.329609e+09  3.090145e+09   
Renault Twingo   5.757505e+11  3.404802e+12  3.358962e+11  6.342587e+10  

In [333]:
#cold.NEDC/hot.NEDC
NEDC_ratio = cold.NEDC/hot.NEDC
WLTP_ratio = cold.WLTP/hot.WLTP

In [334]:
cold.ix[0].NEDC/hot.ix[0].NEDC

C:\Users\Bernat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


pollutant  phase     
CO2        complete      1.05297
           urban         1.12068
           extraurban    1.00265
           300/d         1.29201
           300           1.29286
NOx        complete      3.54141
           urban         6.47057
           extraurban    1.26013
           300/d         11.4029
           300           11.4111
CO         complete      10.3597
           urban         40.8909
           extraurban    1.19352
           300/d         40.3089
           300           40.3354
HC         complete      19.9804
           urban         20.2404
           extraurban    2.28725
           300/d         20.2008
           300           20.2142
PN         complete          NaN
           urban             NaN
           extraurban        NaN
           300/d             NaN
           300               NaN
Name: Audi A1, dtype: object

In [335]:
ratio = pd.concat([NEDC_ratio,WLTP_ratio], axis=1)
ratio

pollutant             CO2                                                NOx  \
phase            complete     urban extraurban     300/d       300  complete   
Audi A1          1.052967  1.120684   1.002651  1.292005  1.292862  3.541407   
BMW 530d         1.043522  1.093781   1.003618  1.063634  1.054279  1.275748   
Audi A3          1.051579  1.112270   1.010104  1.198124  1.199853  0.768135   
Citroen Cactus   1.030752  1.073973   1.001599  1.127386  1.122022  0.397911   
Fiat 500x        1.074304  1.151799   1.026318  1.190871  1.184292  0.767078   
Opel Astra       1.107838  1.187551   1.047885  1.294140  1.300331  1.179540   
Fiat Panda       1.051143  1.110736   1.000771  1.227738  1.226000  1.250426   
Kia Sportage     1.056228  1.114922   1.013596  1.187162  1.186453  1.104749   
Peugeot Partner  1.049275  1.097937   1.017742  1.144688  1.142123  0.929301   
Renault Twingo   1.055607  1.104228   1.014615  1.225855  1.225842  0.857126   
VW Golf          1.090471  1.145398   1.036337  1.167506  1.171498  0.643084   

pollutant                                                      ...      \
phase               urban extraurban      300/d        300     ...       
Audi A1          6.470567   1.260133  11.402857  11.411052     ...       
BMW 530d         3.159198   0.810563   3.690034   3.654872     ...       
Audi A3          0.622021   0.886053   1.386058   1.388518     ...       
Citroen Cactus   0.591350   0.233639   0.882854   0.877852     ...       
Fiat 500x        1.031478   0.735011   3.721434   3.686964     ...       
Opel Astra       1.383560   0.456242   3.340576   3.355429     ...       
Fiat Panda       5.915576   0.517815  32.783390  32.744401     ...       
Kia Sportage     1.459395   0.875661   8.082436   8.077459     ...       
Peugeot Partner  1.175548   0.192025   0.772034   0.773753     ...       
Renault Twingo   1.091724   0.513600   1.981906   1.981439     ...       
VW Golf          0.726537   0.573253   1.348689   1.349041     ...       

pollutant              HC                                  PN              \
phase           extrahigh       300/d         300    complete         low   
Audi A1          0.871358    6.177348    6.198760         NaN         NaN   
BMW 530d         1.187724   11.901153   12.011879    3.845710    7.400410   
Audi A3          0.345523         NaN         NaN  131.633255  138.404565   
Citroen Cactus   0.003809    8.851376    8.784250    2.570842    4.960966   
Fiat 500x        0.783953  309.313204  308.261876   14.924136   42.009588   
Opel Astra       0.955464    5.302095    5.302579    2.150409    5.906542   
Fiat Panda       1.070240    1.545594    1.550811    1.537000    2.220455   
Kia Sportage     1.171414   19.421075   19.447419    1.470780    1.974872   
Peugeot Partner       NaN         NaN         NaN    0.862370    1.051023   
Renault Twingo   1.106897    3.713358    3.686440    3.838267   11.306240   
VW Golf          2.167092    0.915573    0.915434  120.062133  275.032915   

pollutant                                                                  
phase                medium       high  extrahigh       300/d         300  
Audi A1                 NaN        NaN        NaN         NaN         NaN  
BMW 530d           1.445930   0.746221   4.297282   13.516978   13.557770  
Audi A3          133.437455  75.294095  71.799806  129.364051  128.446554  
Citroen Cactus     0.977353   0.969625   0.867246    6.671914    6.660465  
Fiat 500x          0.812990   0.981780   0.890935   76.911489   76.727699  
Opel Astra         2.352972   1.308119   1.111267    7.650978    7.643869  
Fiat Panda         1.061607   0.951503   1.008159    2.289561    2.297835  
Kia Sportage       0.856345   0.922550   0.886730    2.510420    2.521051  
Peugeot Partner    0.731809   0.695529   0.588838    1.138395    1.146936  
Renault Twingo     2.642528   0.672789   0.582089   11.408165   11.325469  
VW Golf            0.945707   1.020877   0.703474  377.970852  378.169545  

[11 r

In [336]:
ratio.to_excel('ratio_coldvshot.xlsx')